In [255]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
import time
from datetime import datetime
from concurrent import futures
from pandas import DataFrame,read_csv
import scrap_config 

result_list=[]
def update_done_keyword_csv(df:DataFrame):
    df=df[["keyword","checking_date"]]
    base_df=read_csv(scrap_config.done_keywords_csv_path)
    base_df=base_df.append(df,ignore_index=True)
    base_df=base_df.drop_duplicates(subset=['keyword'],keep=False)
    base_df.to_csv(scrap_config.done_keywords_csv_path)
    
def save_result():
    final_df=DataFrame(result_list)
    date=datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    file_name=f"report_{date}"
    final_df.to_csv(f"{file_name}.csv",index=False)
    update_done_keyword_csv(final_df)
def get_driver():
    url='https://www.simply.com/en/'
    driver=Chrome(executable_path=r"/home/rifat/Documents/interview/domain_scraping/driver/chromedriver")
    driver.get(url)
    return driver
def is_english_word(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        pass
    else:
        return s
def prepare_keyword_list(df:DataFrame):
    df.columns = map(str.lower, df.columns)
    keywords=list(df.keyword)
    keywords_set=set(map(is_english_word,keywords))
    keywords_set.remove(None)
    keywords_list=list(keywords_set)
    keywords_list=[word.replace(" ","") for word in keywords_list]
    num_of_lists=20
    lists_of_keywords=[keywords_list[x:x+num_of_lists] for x in range(0, len(keywords_list), num_of_lists)]
    return lists_of_keywords
def check_domain_availability(keywords_list):
    global domain_extentions
    global result_list
    driver =get_driver()
    is_first_load=False
    for keyword in keywords_list:
        result_dict={'keyword':keyword}
        for extention in scrap_config.domain_extentions:
            try:
                full_domain=f"{keyword}.{extention}"
                inputElement = driver.find_element_by_class_name("form-control")
                if not is_first_load:
                    inputElement.send_keys(Keys.CONTROL + "a")
                    inputElement.send_keys(Keys.DELETE)
                is_first_load=False
                inputElement.send_keys(full_domain)
                inputElement.send_keys(Keys.ENTER)
                time.sleep(.6)
                elements=driver.find_elements_by_class_name('text-success')
                text=elements[0].text
                if 'The domain you searched for is available!' in text:
                    result_dict[extention]=1
                    # print(full_domain,"-->Domain Available")
                else:
                    result_dict[extention]=0
                    # print(full_domain,"-->Domain not available")
                
                del elements
            except Exception as e:
                result_dict[extention]=str(e)
                print("Exception-->",e)
        result_dict["checking_date"]=datetime.now().strftime("%Y-%m-%d %H-%M-%S")
        result_list.append(result_dict)
    driver.quit()

In [256]:
df=read_csv('google_dk__all-keywords_2022-01-23_11-32-06-2.csv',encoding="latin1")
lists_of_keywords = prepare_keyword_list(df)

In [257]:
t0=time.time()
with futures.ThreadPoolExecutor() as executor: # default/optimized number of threads
  titles = list(executor.map(check_domain_availability, lists_of_keywords))
save_result()
print("Time Taken-->",time.time()-t0)

In [252]:

# final_df=DataFrame(result_list)
final_df

,keyword,nu,com,net,dk,checking_date
0,bilspil,1,0,1,0,2022-01-25 13-20-43
1,bilauktionpoliti,1,1,1,1,2022-01-25 13-20-43
2,wvbil,1,1,1,1,2022-01-25 13-20-47
3,bilkort,1,0,1,1,2022-01-25 13-20-48
4,bilogco,1,0,1,0,2022-01-25 13-20-48
...,...,...,...,...,...,...
180,bedstebrugtbil2020,0,1,1,1,2022-01-25 13-24-12
181,fdmforsikringbil,1,1,1,1,2022-01-25 13-24-17
182,bmwelbil,1,1,1,1,2022-01-25 13-24-21
183,stickerstilbil,1,1,1,1,2022-01-25 13-24-26


In [253]:
final_df2= final_df[['keyword', 'checking_date']]

In [254]:
final_df2.to_csv("done_kwywords.csv",index=False)

In [179]:
isEnglish("how about this one : 通 asfަ")

False